In [1]:
def extract_sequence(string, indices):
    return ''.join([string[i] for i in indices])

# Example usage
string = "023650274221"
indices = [6, 5, 2, 7, 4, 0, 9, 8]

extracted_sequence = extract_sequence(string, indices)
print(extracted_sequence)


20375024
